In [1]:
import pandas as pd
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

In [2]:
# seeds
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 64
EPOCHS = 30
LR = 1e-3

## Prepare Data

In [3]:
# create custom dataset class



class CustomDataset(Dataset):
    def __init__(self, img_dir, annotations_df=None, transform=None):
        """
        img_dir: folder containing images
        annotations_file: CSV path for train labels (None for test set)
        transform: torchvision transforms for the images
        """
        self.img_dir = Path(img_dir)
        self.transform = transform

        if annotations_df:
            self.img_labels = annotations_df.reset_index(drop=True)
            # get class names
            self.classes = sorted(self.img_labels['label'].unique())
            self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
            self.img_labels['target'] = self.img_labels['label'].map(self.class_to_idx)
            self.is_test = False
        else:  # Test set
            self.img_labels = None
            self.is_test = True
            self.classes = None
            self.class_to_idx = None

    def __len__(self):
        if self.is_test:
            return len(list(self.img_dir.glob("*.png")))
        else:
            return len(self.img_labels)

    def __getitem__(self, idx):
        if self.is_test:
            img_name = f"{idx+1}.png"  # test images are named 1.png, 2.png, ...
            label = None
        else:
            img_name = f"{self.img_labels.iloc[idx, 0]}.png"
            label = int(self.img_labels.iloc[idx, 2])  # numeric target

        img_path = self.img_dir / img_name
        image = Image.open(img_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, int(img_name.split('.')[0])  # return image + id for submission
        else:
            return image, label


In [4]:
# CIFAR10 stats
MEAN = (0.4914, 0.4822, 0.4465)
STD = (0.2470, 0.2435, 0.2616)

train_tfms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])
test_tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

# Full train set (with augmentations)
trainset_full = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_tfms)

# --- stratified indices ---
targets = np.array(trainset_full.targets)
train_idx, val_idx = train_test_split(
    np.arange(len(targets)),
    test_size=0.10,            # 10% val
    stratify=targets,
    random_state=42
)

# A second dataset object for val, same underlying data but val/test transforms (no aug)
valset_base = datasets.CIFAR10(root='./data', train=True, download=False, transform=test_tfms)

# Subsets with their own transforms
trainset = Subset(trainset_full, train_idx)  # uses train_tfms
valset   = Subset(valset_base,   val_idx)    # uses test_tfms

# Loaders
g = torch.Generator().manual_seed(42)  # reproducible shuffling
train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, generator=g)
val_loader   = torch.utils.data.DataLoader(valset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Test set (unchanged)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_tfms)
test_loader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# (optional) sanity check
print(f"Train: {len(trainset)}  Val: {len(valset)}  Test: {len(testset)}")

100%|██████████| 170M/170M [00:15<00:00, 10.8MB/s]


Train: 45000  Val: 5000  Test: 10000


## Custom Model class

In [5]:
import torch
import torch.nn as nn
import torchvision.models as models

class CIFAR10ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # no pretrained weights
        self.model = models.resnet18(weights=None)
        # change because we don't want to downsample too much
        self.model.conv1 = nn.Conv2d(
            3, 64, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.model.maxpool = nn.Identity()  # remove maxpool for 32x32 images
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)


## Training loop

In [6]:
# one training epoch
def train_epoch(train_loader, model, criterion, optimizer, device):
    model.train()

    losses = []

    for idx, data in enumerate(train_loader):

        features, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        # print(f"Batch ID: {idx} \t Loss: {loss.item()}")
    return losses

## Validation loop

In [7]:
def evaluate(dataloader, model, criterion, device):
    model.eval() # weights are not updated

    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        losses = []

        for features, labels in dataloader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)

            loss = criterion(outputs, labels) # we need val loss
            losses.append(loss.item())

            _, preds = torch.max(outputs, dim=1)
            total_correct += (preds==labels).sum().item()
            total_samples += labels.size(0)

        accuracy = total_correct / total_samples * 100

    return losses, accuracy

## Training + Validation

In [8]:
def train_modelcv(train_loader, test_loader, model, criterion, optimizer, num_epochs, device):
    train_losses = []
    val_losses = []
    best_measure = 0
    best_epoch = -1

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs-1}")
        print("-" * 10)

        train_loss = train_epoch(train_loader, model, criterion, optimizer, device)
        train_losses.append(train_loss)

        val_loss, measure = evaluate(test_loader, model, criterion, device)
        val_losses.append(val_loss)

        print("Performance measure: ", measure)

        if measure > best_measure:
            best_weights = model.state_dict()
            best_measure = measure
            best_epoch = epoch
            print(f"Current Best is epoch {best_epoch} with {best_measure} %")

    return best_weights, best_measure, best_epoch, train_losses, val_losses

## Configurations

In [9]:
loss = torch.nn.CrossEntropyLoss()

model = CIFAR10ResNet18(10)
model = torch.nn.DataParallel(model)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Model Training

In [10]:
best_weights, best_measure, best_epoch, train_losses, val_losses = train_modelcv(train_loader,
                                                                                 val_loader,
                                                                                 model,
                                                                                 loss,
                                                                                 optimizer,
                                                                                 EPOCHS,
                                                                                 DEVICE)


Epoch 0/29
----------
Performance measure:  60.4
Current Best is epoch 0 with 60.4 %
Epoch 1/29
----------
Performance measure:  65.16
Current Best is epoch 1 with 65.16 %
Epoch 2/29
----------
Performance measure:  77.06
Current Best is epoch 2 with 77.06 %
Epoch 3/29
----------
Performance measure:  80.08
Current Best is epoch 3 with 80.08 %
Epoch 4/29
----------
Performance measure:  82.6
Current Best is epoch 4 with 82.6 %
Epoch 5/29
----------
Performance measure:  79.92
Epoch 6/29
----------
Performance measure:  84.02
Current Best is epoch 6 with 84.02 %
Epoch 7/29
----------
Performance measure:  85.11999999999999
Current Best is epoch 7 with 85.11999999999999 %
Epoch 8/29
----------
Performance measure:  85.88
Current Best is epoch 8 with 85.88 %
Epoch 9/29
----------
Performance measure:  86.53999999999999
Current Best is epoch 9 with 86.53999999999999 %
Epoch 10/29
----------
Performance measure:  87.42
Current Best is epoch 10 with 87.42 %
Epoch 11/29
----------
Performance

In [10]:
import matplotlib.pyplot as plt
def plot_train_val_loss(train_loss, val_loss):
    # get average loss for training and validation
    t_losses = [np.mean(i) for i in train_loss]
    v_losses = [np.mean(i) for i in val_loss]
    plt.plot(t_losses, label='train loss')
    plt.plot(v_losses, label='val loss')
    plt.legend()
    plt.show()

plot_train_val_loss(train_losses, val_losses)


In [17]:
model.load_state_dict(best_weights)
model.eval()
predictions = []
with torch.no_grad():
    for features in test_loader:
        features = features[0].to(DEVICE)
        output = model(features)
        preds = torch.argmax(output, dim=1)
        predictions.extend(preds.cpu().numpy())

image_id = [idx for idx in range(1, len(predictions)+1)]

classes = {0 : 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat',
           4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

predictions = list(map(lambda x: classes[x], predictions))

submission_df = pd.DataFrame({
    "id": image_id,
    "label": predictions
})

submission_df.head()


,id,label
0,1,cat
1,2,ship
2,3,automobile
3,4,airplane
4,5,frog


In [18]:
submission_df.to_csv("cifar_submission1.csv", index=False)